In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

In [18]:
consumption=pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CreditConsumptionData.xlsx")
behaviour =pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CustomerBehaviorData.xlsx")
demographic=pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CustomerDemographics.xlsx")

In [19]:
df=pd.merge(left=behaviour,right=consumption,left_on=consumption['ID'],right_on=behaviour['ID'],how='inner')

df=df.drop('key_0',axis=1)

df=pd.merge(left=df,right=demographic,left_on=df['ID_y'],right_on=demographic['ID'],how='inner')

df=df.drop(['key_0','ID_x','ID_y','ID'],axis=1)

###### Removed column key_0,ID_x, ID_y,ID as it is not required for prediction also  unique identifiers can be redundant 

In [20]:
df_new = df.loc[ df.cc_cons.isna() ]
df_new.drop('cc_cons',axis=1,inplace=True)
df_new.dropna(inplace=True)

In [21]:
# separate the new and existing customers
df = df.loc[ df.cc_cons.notna() ]
df.dropna(inplace=True)

In [22]:
y=df['cc_cons']

In [23]:
df.drop('cc_cons',axis=1,inplace=True)

In [24]:
df_num=df.select_dtypes(exclude ='object').columns

In [25]:
df_cat=df.select_dtypes(include ='object').columns

In [26]:
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler , OneHotEncoder ,StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error , mean_squared_error , r2_score
from sklearn.feature_selection import SelectKBest ,f_classif

In [27]:
def rmspe(train_preds_pipe1, y_train):
    mask = train_preds_pipe1 != 0
    rmspe = np.sqrt(np.mean(np.square((train_preds_pipe1 - y_train) / train_preds_pipe1)[mask] ** 2))
    return rmspe

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [29]:
ct1 = ColumnTransformer(
    transformers=[
    ('olt',RobustScaler(),df_num),
    ('Ohe',OneHotEncoder(drop='first'),df_cat)
                    ])

In [30]:
lr=LinearRegression()
rfr = RandomForestRegressor(max_depth=3)
knr=KNeighborsRegressor(n_neighbors=5,weights='uniform',metric='euclidean')
svr = SVR(kernel='poly',C=0.1)

In [32]:
pipe1=Pipeline(steps=[
    ('col_transformation',ct1),
    ('Feature Selection',SelectKBest(f_classif,k=25)),
    ('linear_regression',lr)
])

In [33]:
pipe2=Pipeline(steps=[
    ('col_transformation',ct1),
    ('Feature Selection',SelectKBest(f_classif,k=25)),
    ('Random Forest',rfr)
])

In [34]:
pipe3=Pipeline(steps=[
    ('col_transformation',ct1),
    ('Feature Selection',SelectKBest(f_classif,k=25)),
    ('KNN Regressor',knr)
])

In [35]:
pipe4=Pipeline(steps=[
    ('col_transformation',ct1),
    ('Feature Selection',SelectKBest(f_classif,k=25)),
    ('Support Vector Machine',svr)
])

In [36]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object')),
                                                 ('Ohe',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['loan_enq', 'account_type', 'gender', 'Income'], dtype='object'))])),
                ('Feature Selection', SelectKBest(k=25)),
                ('linear_regression', LinearRegression())])

In [37]:
train_preds_pipe1 = pipe1.predict(X_train)
test_preds_pipe1 = pipe1.predict(X_test)

In [38]:
print("MAE train pipe1 :: ",mean_absolute_error(train_preds_pipe1, y_train))
print("MAE test pipe1 :: ",mean_absolute_error(test_preds_pipe1, y_test))
print("MAPE train pipe1 :: ",mean_absolute_percentage_error(train_preds_pipe1, y_train))
print("MAPE test pipe1 :: ",mean_absolute_percentage_error(test_preds_pipe1, y_test))
print("MSE train pipe1 :: ",mean_squared_error(train_preds_pipe1, y_train))
print("MSE test pipe1 :: ",mean_squared_error(test_preds_pipe1, y_test))
print("RMSPE train pipe1 :: ",rmspe(train_preds_pipe1,y_train))
print("RMSPE test pipe1 :: ",rmspe(test_preds_pipe1,y_test))

MAE train pipe1 ::  71088.47426162293
MAE test pipe1 ::  69126.31597305332
MAPE train pipe1 ::  1.263878234964714
MAPE test pipe1 ::  1.2384571963186604
MSE train pipe1 ::  9725650328.449469
MSE test pipe1 ::  9126963261.295912
RMSPE train pipe1 ::  7.881141602394455
RMSPE test pipe1 ::  7.9567827002392315


In [40]:
pipe2.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object')),
                                                 ('Ohe',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['loan_enq', 'account_type', 'gender', 'Income'], dtype='object'))])),
                ('Feature Selection', SelectKBest(k=25)),
                ('Random Forest', RandomForestRegressor(max_depth=3))])

In [43]:
train_preds_pipe2 = pipe2.predict(X_train)
test_preds_pipe2 = pipe2.predict(X_test)

In [44]:
print("MAE train pipe2 :: ",mean_absolute_error(train_preds_pipe2, y_train))
print("MAE test  pipe2 :: ",mean_absolute_error(test_preds_pipe2, y_test))
print("RMAPE train  pipe2 :: ",np.sqrt(mean_absolute_percentage_error(train_preds_pipe2, y_train)))
print("RMAPE test  pipe2 :: ",np.sqrt(mean_absolute_percentage_error(test_preds_pipe2, y_test)))
print("RMSPE train pipe2 :: ",rmspe(train_preds_pipe2,y_train))
print("RMSPE test pipe2 :: ",rmspe(test_preds_pipe2,y_test))

MAE train pipe2 ::  70832.91263068492
MAE test  pipe2 ::  69119.31663143326
RMAPE train  pipe2 ::  1.118126448201514
RMAPE test  pipe2 ::  1.1091730829190087
RMSPE train pipe2 ::  7.355277100589023
RMSPE test pipe2 ::  7.264629723456522


In [45]:
pipe3.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object')),
                                                 ('Ohe',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['loan_enq', 'account_type', 'gender', 'Income'], dtype='object'))])),
                ('Feature Selection', SelectKBest(k=25)),
                ('KNN Regressor', KNeighborsRegressor(metric='euclidean'))])

In [46]:
train_preds_pipe3 = pipe3.predict(X_train)
test_preds_pipe3 = pipe3.predict(X_test)

In [47]:
print("MAE train pipe3 :: ",mean_absolute_error(train_preds_pipe3, y_train))
print("MAE test  pipe3 :: ",mean_absolute_error(test_preds_pipe3, y_test))
print("MAPE train  pipe3 :: ",mean_absolute_percentage_error(train_preds_pipe3, y_train))
print("MAPE test  pipe3 :: ",mean_absolute_percentage_error(test_preds_pipe3, y_test))
print("RMSPE train pipe3 :: ",rmspe(train_preds_pipe3,y_train))
print("RMSPE test pipe3 :: ",rmspe(test_preds_pipe3,y_test))

MAE train pipe3 ::  58769.216784469885
MAE test  pipe3 ::  70284.06647807638
MAPE train  pipe3 ::  0.9268715460056908
MAPE test  pipe3 ::  2.2399258660450854
RMSPE train pipe3 ::  3.0656016696214965
RMSPE test pipe3 ::  217.8040229200952


In [48]:
pipe4.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object')),
                                                 ('Ohe',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['loan_enq', 'account_type', 'gender', 'Income'], dtype='object'))])),
                ('Feature Selection', SelectKBest(k=25)),
                ('Support Vector', SVR(C=0.1, kernel='poly'))])

In [89]:
train_preds_pipe4 = pipe4.predict(X_train)
test_preds_pipe4 = pipe4.predict(X_test)

In [91]:
print("MAE train pipe4 :: ",mean_absolute_error(train_preds_pipe4, y_train))
print("MAE test  pipe4 :: ",mean_absolute_error(test_preds_pipe4, y_test))
print("MAPE train  pipe4 :: ",mean_absolute_percentage_error(train_preds_pipe4, y_train))
print("MAPE test  pipe4 :: ",mean_absolute_percentage_error(test_preds_pipe4, y_test))
print("RMSPE train pipe4 :: ",rmspe(train_preds_pipe4,y_train))
print("RMSPE test pipe4 :: ",rmspe(test_preds_pipe4,y_test))

MAE train pipe4 ::  49553.489586641444
MAE test  pipe4 ::  46243.944793321236
MAPE train  pipe4 ::  3.690462845917078
MAPE test  pipe4 ::  3.4381815455684426
RMSPE train pipe4 ::  179.18376189909955
RMSPE test pipe4 ::  168.11926692822576


In [135]:
i=1
for pipe in [pipe1,pipe2,pipe3,pipe4]:
    pipe.fit(X_train,y_train)
    train_preds_pipe = pipe.predict(X_train)
    test_preds_pipe = pipe.predict(X_test)    
    print(f"RMSPE train {pipe.__class__.__name__}{i} :: ", rmspe(train_preds_pipe, y_train))
    print(f"RMSPE test {pipe.__class__.__name__}{i}:: ", rmspe(test_preds_pipe, y_test))
    print(" ")
    i+=1

RMSPE train Pipeline1 ::  7.881141602394455
RMSPE test Pipeline1::  7.9567827002392315
 
RMSPE train Pipeline2 ::  7.426635865941169
RMSPE test Pipeline2::  7.306940039723083
 
RMSPE train Pipeline3 ::  3.0656016696214965
RMSPE test Pipeline3::  217.8040229200952
 
RMSPE train Pipeline4 ::  179.18376189909955
RMSPE test Pipeline4::  168.11926692822576
 


#### From the above RMSPE we can see that Pipeline 1(Linear Regression) and Pipeline 2(RandomForestRegressor) as it has the lowest RMSPE values for both the training and test .Checking other metrices.

In [137]:
print("MAE train pipe2 :: ",mean_absolute_error(train_preds_pipe2, y_train))
print("MAE test  pipe2 :: ",mean_absolute_error(test_preds_pipe2, y_test))
print("MAPE train  pipe2 :: ",mean_absolute_percentage_error(train_preds_pipe2, y_train))
print("MAPE test  pipe2 :: ",mean_absolute_percentage_error(test_preds_pipe2, y_test))
print("MSE train pipe2 :: ",mean_squared_error(train_preds_pipe2, y_train))
print("MSE test pipe2 :: ",mean_squared_error(test_preds_pipe2, y_test))
print("RMSPE train pipe2 :: ",rmspe(train_preds_pipe2,y_train))
print("RMSPE test pipe2 :: ",rmspe(test_preds_pipe2,y_test))

MAE train pipe2 ::  70763.74492455728
MAE test  pipe2 ::  69112.37173727513
MAPE train  pipe2 ::  1.2508613103068473
MAPE test  pipe2 ::  1.2314621234144354
MSE train pipe2 ::  9616699666.74927
MSE test pipe2 ::  9092487910.086346
RMSPE train pipe2 ::  7.387896183627704
RMSPE test pipe2 ::  7.282106455266152


In [39]:
print("MAE train pipe1 :: ",mean_absolute_error(train_preds_pipe1, y_train))
print("MAE test pipe1 :: ",mean_absolute_error(test_preds_pipe1, y_test))
print("MAPE train pipe1 :: ",mean_absolute_percentage_error(train_preds_pipe1, y_train))
print("MAPE test pipe1 :: ",mean_absolute_percentage_error(test_preds_pipe1, y_test))
print("MSE train pipe1 :: ",mean_squared_error(train_preds_pipe1, y_train))
print("MSE test pipe1 :: ",mean_squared_error(test_preds_pipe1, y_test))
print("RMSPE train pipe1 :: ",rmspe(train_preds_pipe1,y_train))
print("RMSPE test pipe1 :: ",rmspe(test_preds_pipe1,y_test))

R2 score train pipe1 ::  -212.34808059446073
R2 score test pipe1 ::  -194.94283438790725
MAE train pipe1 ::  71088.47426162293
MAE test pipe1 ::  69126.31597305332
MAPE train pipe1 ::  1.263878234964714
MAPE test pipe1 ::  1.2384571963186604
MSE train pipe1 ::  9725650328.449469
MSE test pipe1 ::  9126963261.295912
RMSPE train pipe1 ::  7.881141602394455
RMSPE test pipe1 ::  7.9567827002392315


#### By looking at the other metrices we can conclude that pipe 2 is performing better than pipe 1, therfore using pipe 2 for making predictions.

In [150]:
pred=pd.DataFrame(pipe2.predict(df_new))

In [151]:
df_new=df_new.reset_index(drop = True)

In [152]:
df_new['pred_cc_cons']=pred

In [155]:
df_new

,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,...,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction,pred_cc_cons,cc_cons
0,2795.450,1290.00,6141.05,676.50,9146.10,15479.00,4.0,10,4,47,...,M,37,MEDIUM,11.9,4,575.0,1,16.0,54815.547962,9105.930
1,29405.130,1640.00,1670.00,2463.92,10947.50,956.00,3.0,10,4,50,...,M,33,MEDIUM,7.8,6,394.0,0,19.0,54470.435001,88877.540
2,5985.200,6189.00,2696.09,766.00,716.00,4133.22,7.0,30,5,24,...,M,53,LOW,33.0,10,324.0,1,15.0,57357.348023,53428.650
3,2105.930,18225.00,34763.56,167.00,4260.27,20185.00,1.0,99,71,5,...,M,33,MEDIUM,7.8,5,370.0,1,7.0,64191.680867,58575.490
4,3269.000,3532.00,3158.40,2699.77,3373.48,5120.00,2.0,10,47,5,...,M,62,LOW,12.6,9,505.0,1,3.0,58144.910525,35922.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,3043.000,742.20,3418.00,2181.94,12032.60,788.00,1.0,5,6,1,...,M,60,MEDIUM,24.0,5,968.0,0,9.0,57595.190633,8804.760
4989,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,4,1,...,M,35,MEDIUM,13.5,1,523.0,0,7.0,54637.939288,76999.670
4990,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,50,18,...,M,53,MEDIUM,9.9,7,723.0,1,2.0,54958.836062,15961.000
4991,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,1,1,...,M,62,HIGH,37.8,8,863.0,1,16.0,56237.881298,51021.825
